# Load model

In [11]:
%pip install dotenv boto3 transformers torch --quiet
print("Installation complete")

You should consider upgrading via the '/Users/nlembers/Documents/projects/openshift-ai-huggingface/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Installation complete


In [19]:
import os
import dotenv

dotenv.load_dotenv()

S3_ENDPOINT = os.getenv("S3_ENDPOINT")
S3_BUCKET = "modelstorage"
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

MODEL_NAME = "Qwen/Qwen3-0.6B"
MODEL_DIR = "qwen3"

In [ ]:
import boto3
from transformers import AutoModel, AutoTokenizer
import os

s3_client = boto3.client(
    's3',
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print(f"Loading model: {MODEL_NAME}")
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

notebook_dir = os.getcwd()
model_dir = os.path.join(notebook_dir, "models")
model_path = os.path.join(model_dir, MODEL_DIR)
os.makedirs(model_path, exist_ok=True)

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading model: Qwen/Qwen3-0.6B
Saving model locally...


('/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/tokenizer_config.json',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/special_tokens_map.json',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/chat_template.jinja',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/vocab.json',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/merges.txt',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/added_tokens.json',
 '/Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/tokenizer.json')

In [ ]:
from boto3.s3.transfer import TransferConfig
import os.path

# multipart_threshold: Files larger than 10MB will use multipart upload
config = TransferConfig(
    multipart_threshold=10 * 1024 * 1024,  # 10MB
    multipart_chunksize=10 * 1024 * 1024,  # 10MB per part
    use_threads=True
)

for root, dirs, files in os.walk(model_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        relative_path = os.path.relpath(local_file_path, model_path)
        s3_key = f"{MODEL_DIR}/{relative_path}"
        
        file_size = os.path.getsize(local_file_path)
        file_size_mb = file_size / (1024 * 1024)
        
        print(f"  Uploading: {s3_key} ({file_size_mb:.2f} MB)")
        
        try:
            s3_client.upload_file(
                local_file_path, 
                S3_BUCKET, 
                s3_key,
                Config=config
            )
            print(f"Uploaded successfully")
        except Exception as e:
            print(f"Error uploading: {e}")
            raise

print(f"\nModel successfully uploaded to s3://{S3_BUCKET}/{MODEL_DIR}/")

  Uploading: qwen3/model.safetensors


S3UploadFailedError: Failed to upload /Users/nlembers/Documents/projects/openshift-ai-huggingface/models/qwen3/model.safetensors to modelstorage/qwen3/model.safetensors: An error occurred (InternalError) when calling the UploadPart operation (reached max retries: 4): We encountered an internal error. Please try again.